# Vault Shock Simulator

This toy model stress-tests a lending vault under a sudden collateral price shock.

It has two parts:
1) **Solvency / liquidation stress**: does forced liquidation create **bad debt**?
2) **Liquidity / withdrawals stress**: can the vault meet a withdrawal run without disorderly unwinds?


In [17]:
# ---- Inputs ----

TVL = 100_000_000          # total vault TVL in USD
exposure_share = 0.20      # % of TVL exposed to the affected market (0.20 = 20%)

# Shock + liquidation mechanics
shock = 0.35               # collateral price drop (0.35 = -35%)
# Haircut model (stress-dependent)
h0 = 0.04        # baseline haircut in normal conditions (fees + mild slippage)
h_max = 0.30     # maximum haircut under extreme stress
k_h = 6.0        # sensitivity: higher = haircut ramps faster with shock

# Simplified leverage setup (average pre-shock borrower LTV vs LLTV)
LLTV = 0.80                # liquidation threshold
avg_LTV_pre = 0.75         # average LTV before the shock among exposed borrowers

# Liquidity / withdrawals
L0 = 8_000_000             # instantly available liquidity (idle)
L1 = 25_000_000            # withdrawable within horizon T (e.g., 1 day) under normal conditions
d = 0.65                   # stress discount factor on L1 (0.65 means only 65% is realistically available)
run_r = 0.20               # withdrawal run size as % of TVL (0.20 = 20%)

# Borrower LTV distribution assumptions
N = 30_000          # number of simulated borrowers (keep 10k–50k for iPad speed)
ltv_mean = 0.60     # assumed average borrower LTV pre-shock
ltv_std  = 0.12     # assumed dispersion of borrower LTVs pre-shock

In [18]:
import random

def beta_params_from_mean_std(mean, std):
    # Convert mean/std on [0,1] into Beta(alpha,beta)
    var = std**2

    # Beta variance must be < mean*(1-mean)
    max_var = mean * (1 - mean) - 1e-9
    if var >= max_var:
        var = max_var

    k = mean * (1 - mean) / var - 1
    alpha = mean * k
    beta = (1 - mean) * k
    return alpha, beta

alpha, beta = beta_params_from_mean_std(ltv_mean, ltv_std)

def liquidation_fraction_mc(shock, LLTV, N, alpha, beta):
    # Liquidation condition: LTV_pre >= LLTV*(1-shock)
    threshold = LLTV * (1 - shock)
    liquidated = 0
    for _ in range(N):
        ltv_pre = random.betavariate(alpha, beta)
        if ltv_pre >= threshold:
            liquidated += 1
    return liquidated / N, threshold

liq_frac, ltv_threshold = liquidation_fraction_mc(shock, LLTV, N, alpha, beta)

liq_frac, ltv_threshold

(0.7479666666666667, 0.52)

In [19]:
import random

def beta_params_from_mean_std(mean, std):
    # Convert mean/std on [0,1] into Beta(alpha,beta)
    var = std**2
    max_var = mean * (1 - mean) - 1e-9
    if var >= max_var:
        var = max_var

    k = mean * (1 - mean) / var - 1
    alpha = mean * k
    beta = (1 - mean) * k
    return alpha, beta

alpha, beta = beta_params_from_mean_std(ltv_mean, ltv_std)

def liquidation_stats_mc(shock, LLTV, N, alpha, beta):
    """
    Returns:
    - liquidation fraction
    - avg pre-shock LTV of liquidated borrowers
    - liquidation threshold (pre-shock)
    """
    threshold = LLTV * (1 - shock)
    liquidated_ltvs = []

    for _ in range(N):
        ltv_pre = random.betavariate(alpha, beta)
        if ltv_pre >= threshold:
            liquidated_ltvs.append(ltv_pre)

    if len(liquidated_ltvs) == 0:
        return 0.0, 0.0, threshold

    liq_frac = len(liquidated_ltvs) / N
    avg_ltv_liq_pre = sum(liquidated_ltvs) / len(liquidated_ltvs)

    return liq_frac, avg_ltv_liq_pre, threshold

liq_frac, avg_ltv_liq_pre, ltv_threshold = liquidation_stats_mc(
    shock, LLTV, N, alpha, beta
)

avg_ltv_liq_post = avg_ltv_liq_pre / (1 - shock)

liq_frac, avg_ltv_liq_pre, avg_ltv_liq_post, ltv_threshold

(0.7467666666666667, 0.6521769120342129, 1.0033490954372506, 0.52)

In [20]:
import math

def haircut_from_shock(shock, h0, h_max, k):
    # bounded, increasing, convex-ish for small shocks
    return h0 + (h_max - h0) * (1 - math.exp(-k * shock))

haircut = haircut_from_shock(shock, h0, h_max, k_h)
haircut

0.2681613286542247

In [21]:
# ---- Solvency / liquidation module ----

exposure_USD = TVL * exposure_share

# liquidation share is now computed endogenously via Monte Carlo (liq_frac)
liquidation_volume_USD = exposure_USD * liq_frac

avg_LTV_post = avg_ltv_liq_post
debt_USD = liquidation_volume_USD * avg_LTV_post

# Net liquidation proceeds after haircut (slippage/fees/incentives)
proceeds_USD = liquidation_volume_USD * (1 - haircut)

bad_debt_USD = max(0.0, debt_USD - proceeds_USD)
loss_share_of_TVL = bad_debt_USD / TVL

(exposure_USD, liquidation_volume_USD, avg_LTV_post, proceeds_USD, bad_debt_USD, loss_share_of_TVL)

(20000000.0,
 14935333.333333334,
 1.0033490954372506,
 10930254.502772937,
 4055098.6872808803,
 0.0405509868728088)

In [22]:
# ---- Liquidity / withdrawals module ----

required_redemptions = run_r * TVL
available_liquidity = L0 + d * L1

liquidity_pass = available_liquidity >= required_redemptions
liquidity_gap = max(0.0, required_redemptions - available_liquidity)

(required_redemptions, available_liquidity, liquidity_pass, liquidity_gap)

(20000000.0, 24250000.0, True, 0.0)

In [23]:
def usd(x): 
    return f"${x:,.0f}"

print("=== Solvency / Liquidations ===")
print("Vault TVL:", usd(TVL))
print("Exposure in affected market:", usd(exposure_USD), f"({exposure_share:.0%} of TVL)")
print("Liquidation volume:", usd(liquidation_volume_USD))
print("Avg LTV post-shock:", f"{avg_LTV_post:.2f}")
print("Net liquidation proceeds:", usd(proceeds_USD))
print("Estimated bad debt:", usd(bad_debt_USD))
print("Loss as % of TVL:", f"{loss_share_of_TVL:.2%}")

print("\n=== Liquidity / Withdrawals ===")
print("Run scenario (withdrawals):", f"{run_r:.0%} of TVL =", usd(required_redemptions))
print("Available liquidity:", usd(available_liquidity), f"(L0 + d·L1 with d={d})")
print("Pass liquidity test?", "✅ YES" if liquidity_pass else "❌ NO")
if not liquidity_pass:
    print("Liquidity gap:", usd(liquidity_gap))

=== Solvency / Liquidations ===
Vault TVL: $100,000,000
Exposure in affected market: $20,000,000 (20% of TVL)
Liquidation volume: $14,935,333
Avg LTV post-shock: 1.00
Net liquidation proceeds: $10,930,255
Estimated bad debt: $4,055,099
Loss as % of TVL: 4.06%

=== Liquidity / Withdrawals ===
Run scenario (withdrawals): 20% of TVL = $20,000,000
Available liquidity: $24,250,000 (L0 + d·L1 with d=0.65)
Pass liquidity test? ✅ YES
